In [1]:
import numpy as np
import xmltodict
import json
import os
import pickle

In [ ]:
# everytime read 5 000 000 lines of xml file and transfer it to json(dictionsry)
num = 0
count = 0
inside = []
name = "./data/output_event_"
with open("./output_events.xml", 'rt') as f:
    a = f.readline()
    b = f.readline()
    for line in f.readlines():
        inside.append(line)
        num += 1
        if num % 5000000 == 0:
            count += 1
            xml = []
            xml.append(a)
            xml.append(b)
            xml.extend(inside)
            if inside[-1] != "</events>":
                xml.append("</events>")
            xml_file = ''.join(xml)
            js = xmltodict.parse(xml_file, encoding='utf-8')
            file_name = name + str(count) + "_" + str(num) + ".json"
            with open(file_name, 'w') as fw:
                json.dump(js, fw)
            inside = []
            del js, xml_file


In [ ]:
# The last time transforming may not be 5 000 000, it is needed to be dealed separately
count += 1
xml = []
xml.append(a)
xml.append(b)
xml.extend(inside)
if inside[-1] != "</events>":
    xml.append("</events>")
xml_file = ''.join(xml)
js = xmltodict.parse(xml_file, encoding='utf-8')
file_name = name + str(count) + "_" + str(num) + ".json"
with open(file_name, 'w') as fw:
    json.dump(js, fw)

In [ ]:
# get all the output_event.json file name and remove .DS_Store file which is automately created by Mac OS
events = os.listdir("./data")
events.remove(".DS_Store")
ordered_events = [0] * len(events)

In [ ]:
# order the event 
for event in events:
    idx = int(event.split("_")[2]) - 1
    ordered_events[idx] = event

In [ ]:
# read all event by json form into memory
eventList = []
for event in ordered_events:
    path = "./data/" + event
    with open(path, 'r') as f:
        json_event = json.load(f)
    eventList.extend(json_event["events"]["event"])

In [ ]:
# pre-process data(remove all the public transportation dataset)
# event is recorded in this form {{"time1":time1, "personId":person, "linkId":"link"}, ...}
eventsWithoutPublic = []
for item in eventList:
    if item.__contains__("@link"):
        if item.__contains__("@person"): 
            if not item["@person"].startswith("pt"):
                eventsWithoutPublic.append({"time":float(item["@time"]), "person":item["@person"], "link":item["@link"]})
        else:
            if item.__contains__("@vehicle"):
                if not item["@vehicle"].startswith("pt"):
                    eventsWithoutPublic.append({"time":float(item["@time"]), "person":item["@vehicle"], "link":item["@link"]})
    

In [ ]:
# store event_without_public.json file
with open("event_without_public.json", 'w') as ewp:
    json.dump(eventsWithoutPublic, ewp)

In [ ]:
# load event_without_public.json file
with open("./event_without_public.json", 'r') as fr:
    eventsWithoutPublic = json.load(fr)

In [ ]:
# divide the event 
quarter = int(len(eventsWithoutPublic) / 4)
eventsWithoutPublic_first = eventsWithoutPublic[:quarter]
eventsWithoutPublic_second = eventsWithoutPublic[quarter : quarter * 2]
eventsWithoutPublic_third = eventsWithoutPublic[quarter*2 : quarter * 3]
eventsWithoutPublic_fourth = eventsWithoutPublic[quarter*3 :]

In [ ]:
def sorted_event_by_link(events):
    # sort data into links
    #  data form is {"link1" : {time1 : {"person1", "person2", ...} ... }, ...}
    linkDict = {}
    e = 0
    for item in events:
        e += 1
        if linkDict.__contains__(item["link"]):
            if linkDict[item["link"]].__contains__(item["time"]):
                (linkDict[item["link"]][item["time"]]).add(item["person"])
            else:
                linkDict[item["link"]][item["time"]] = {item["person"]}
        else:
            linkDict[item["link"]] = {item["time"] : {item["person"]}}
        if e % 10000000 == 0:
            print(e)
    del events
    return linkDict

In [ ]:
linkDict = sorted_event_by_link(eventsWithoutPublic_first)
with open("./link_first.pkl", 'wb') as fsw:
    pickle.dump(linkDict, fsw)

In [ ]:
linkDict = sorted_event_by_link(eventsWithoutPublic_second)
with open("./link_secong.pkl", 'wb') as fsw:
    pickle.dump(linkDict, fsw)

In [ ]:
linkDict = sorted_event_by_link(eventsWithoutPublic_third)
with open("./link_third.pkl", 'wb') as ftw:
    pickle.dump(linkDict, ftw)

In [ ]:
linkDict = sorted_event_by_link(eventsWithoutPublic_fourth)
with open("./link_fourth.pkl", 'wb') as fw:
    pickle.dump(linkDict, fw)

In [ ]:
# with open("./link_fourth.pkl", 'rb') as fr:
#     b = pickle.load(fr)

In [ ]:
with open("./network.xml", 'r') as fn:
    network_xml = fn.read()
    network = xmltodict.parse(network_xml, encoding='utf-8')

In [ ]:
network["network"]["links"]["link"][0]

In [ ]:
network["network"]["links"]["link"][1]["@id"]

In [ ]:

links = dict()
for link in network["network"]["links"]["link"]:
    id = link["@id"]
    links[id] = None

In [ ]:
len(links.keys())

In [ ]:
with open("link_first.pkl", "rb") as flf:
    link_first = pickle.load(flf)
    

In [ ]:
with open("link_second.pkl", "rb") as fsf:
    link_second = pickle.load(fsf)

In [ ]:
with open("link_third.pkl", "rb") as ftf:
    link_third = pickle.load(ftf)

In [ ]:
with open("link_fourth.pkl", "rb") as fff:
    link_fourth = pickle.load(fff)

In [ ]:
stride = int(len(links.keys()) / 10)
# stride = 100
links_key = list(links.keys())

In [ ]:
links_0 = {key: dict() for key in links_key[:stride]}
links_1 = {key: dict() for key in links_key[stride : stride * 2]}
links_2 = {key: dict() for key in links_key[stride * 2 : stride * 3]}
links_3 = {key: dict() for key in links_key[stride * 3 : stride * 4]}
links_4 = {key: dict() for key in links_key[stride * 4 : stride * 5]}
links_5 = {key: dict() for key in links_key[stride * 5 : stride * 6]}
links_6 = {key: dict() for key in links_key[stride * 6 : stride * 7]}
links_7 = {key: dict() for key in links_key[stride * 7 : stride * 8]}
links_8 = {key: dict() for key in links_key[stride * 8 : stride * 9]}
links_9 = {key: dict() for key in links_key[stride * 9 :]}


In [ ]:
link_dict_all = [link_first, link_second, link_third, link_fourth]
link_target_all = [links_0, links_1, links_2, links_3, links_4, links_5, links_6, links_7, links_8, links_9]

In [ ]:
len(links_0.keys())

In [ ]:
idx = 0
for link_target in link_target_all:
    for key, val in link_target.items():
        for link_dict in link_dict_all:
            if link_dict.__contains__(key):
                for time, person in link_dict[key].items():
                    if link_target[key] is not None and link_target[key].__contains__(time):
                        link_target[key][time].update(person)
                    else:
                        link_target[key][time] = person
        print("choose one from four")
    file_name = "link_" + str(idx) + ".pkl"
    print("the " + str(idx) + " link file begin to store")
    with open(file_name, "wb") as link_file:
        pickle.dump(link_target, link_file)
    print("the " + str(idx) + " link file stored")
    idx += 1

In [ ]:
with open("link_0.pkl", 'rb') as fr0:
    l_0 = pickle.load(fr0)

In [ ]:
list(l_0.keys())[:10]

In [ ]:
num = 0
for k, v in l_0.items():
    for i,j in l_0[k].items():
        if len(j) >= 3:
            num +=1
num

In [ ]:
# get the contact data
# {'person1':{'person1':'n1', 'person2':'n2', 'person3':'n3'}, 'person2':{...}}

def contact(linkDict, personContact, stride):
    # stride = 500
    # v {0.0: {'95187201', '123413'}, 3597.0: {'95187201'}, 25549.0: {'322233201'} 
    # {time:{person}}
    for k, v in linkDict.items():
        # {0.0, 3597.0}
        times = list(v.keys())
        # diff = list(map(lambda x : abs(x[1] - x[0]), zip(times[:-1], times[1:])))
        for t1 in range(len(times)):
            for t2 in range(1, len(times)):
                if abs(times[t2] - times[t1]) <= stride:
                    mPersons = list(linkDict[k][times[t1]])
                    cPerson = list(linkDict[k][times[t2]])
                    for p in mPersons:
                        for cp in cPerson:
                            if p == cp:
                                continue
                            if not personContact.__contains__(p):
                                personContact[p] = dict()
                                if not personContact[p].__contains__(cp):
                                    personContact[p][cp] = 0
                                else:
                                    num = personContact[p][cp] + 1
                                    personContact[p][cp] = num
                            else:
                                if not personContact[p].__contains__(cp):
                                    personContact[p][cp] = 0
                                else:
                                    num = personContact[p][cp] + 1
                                    personContact[p][cp] = num
                else:
                    break
    return personContact



In [ ]:
dir = "../event_sorted_by_link/"
link_pkl_list = os.listdir(dir)
personContact = dict()
stride = 500
idx = 0

for file in link_pkl_list:
    print('begin to load data')
    with open(dir + file, "rb") as fr:
        linkDict = pickle.load(fr)
    print("begin to get contact data")
    personContact = contact(linkDict, personContact, stride)
    print("get the contact data in link " + str(idx))
    idx += 1
    
    

In [ ]:
with open("../contact_data.pkl", 'wb') as fwb:
    pickle.dump(personContact, fwb)

In [ ]:
list(personContact.keys())[:10]